<a href="https://colab.research.google.com/github/AleksandraOD/classification-apmath-21/blob/main/Task6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [41]:
!pip install pymorphy2

In [56]:
import pandas as pd
import numpy as np
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
import nltk
import pymorphy2
from sklearn.preprocessing import LabelEncoder

morph = pymorphy2.MorphAnalyzer()

1. Обработать данные, получив для каждого текста набор токенов
2. Обработать токены с помощью pymorphy2

In [84]:
train_doc = "/content/drive/MyDrive/Colab Notebooks/classification-apmath-21/data/news/news_train.txt"
test_doc = "/content/drive/MyDrive/Colab Notebooks/classification-apmath-21/data/news/news_test.txt"

In [85]:
train_data = pd.read_csv(train_doc, sep="\t", names=['theme', 'headline', 'info'])
test_data = pd.read_csv(test_doc, sep="\t", names=['theme', 'headline', 'info'])

In [45]:
train_data.head()

,theme,headline,info
0,sport,Овечкин пожертвовал детской хоккейной школе ав...,Нападающий «Вашингтон Кэпиталз» Александр Овеч...
1,culture,Рекордно дорогую статую майя признали подделкой,"Власти Мексики объявили подделкой статую майя,..."
2,science,Samsung представила флагман в защищенном корпусе,Южнокорейская Samsung анонсировала защищенную ...
3,sport,С футболиста «Спартака» сняли четырехматчевую ...,Контрольно-дисциплинарный комитет (КДК) РФС сн...
4,media,Hopes & Fears объединится с The Village,Интернет-издание Hopes & Fears объявило о свое...


In [46]:
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [47]:
def tokenize_words(input):

  input = input.lower()
  filtered = []

  tokenizer = RegexpTokenizer(r'\w+')
  tokens = tokenizer.tokenize(input)

  for word in tokens:
    word = morph.parse(word)[0].normal_form
    if word not in stopwords.words('russian'):
      filtered.append(word)

  result = " ".join(filtered)

  return result




In [87]:
for i in range(len(train_data)):
  train_data["headline"][i] = tokenize_words(train_data["headline"][i])
  train_data["info"][i] = tokenize_words(train_data["info"][i])

In [50]:
# train_data["headline"][1] = tokenize_words(train_data["headline"][1])

In [88]:
train_data.head()

,theme,headline,info
0,sport,овечкин пожертвовать детский хоккейный школа а...,нападать вашингтон кэпиталзти александр овечки...
1,culture,рекордно дорогой статуя майя признать подделка,власть мексика объявить подделка статуя майя п...
2,science,samsung представить флагман защитить корпус,южнокорейский samsung анонсировать защитить ве...
3,sport,футболист спартак снять четырехматчевой дисква...,контрольный дисциплинарный комитет кдк рфс сня...
4,media,hopes fears объединиться the village,интернет издание hopes fears объявить свой сли...


In [93]:
label_encoder = LabelEncoder()
train_data['target'] = label_encoder.fit_transform(train_data['theme'])
test_data['target'] = label_encoder.transform(test_data['theme'])


In [94]:
train_data.head()

,theme,headline,info,target
0,sport,овечкин пожертвовать детский хоккейный школа а...,нападать вашингтон кэпиталзти александр овечки...,7
1,culture,рекордно дорогой статуя майя признать подделка,власть мексика объявить подделка статуя майя п...,1
2,science,samsung представить флагман защитить корпус,южнокорейский samsung анонсировать защитить ве...,6
3,sport,футболист спартак снять четырехматчевой дисква...,контрольный дисциплинарный комитет кдк рфс сня...,7
4,media,hopes fears объединиться the village,интернет издание hopes fears объявить свой сли...,5


In [58]:
train_data.to_csv( "/content/drive/MyDrive/Colab Notebooks/classification-apmath-21/data/news/filtered_train.txt", index=False, encoding='utf-8-sig')
test_data.to_csv( "/content/drive/MyDrive/Colab Notebooks/classification-apmath-21/data/news/filtered_test.txt", index=False, encoding='utf-8-sig')

3. Обучить word embeddings (fastText, word2vec, gloVe) на тренировочных данных. Можно использовать gensim . 
4. Продемонстрировать семантические ассоциации.

In [95]:
from gensim.models import Word2Vec

In [96]:
train = train_data["info"]
test = test_data["info"]

In [97]:
words = [text.split() for text in train]
w2v = Word2Vec(sentences = words, min_count = 0)

In [98]:
w2v.wv.most_similar('спорт', topn=5)  

[('р', 0.8418865203857422),
 ('экспресс', 0.8301019072532654),
 ('мутко', 0.7627315521240234),
 ('bild', 0.7542246580123901),
 ('футбол', 0.7366930246353149)]

In [99]:
w2v.wv.most_similar('культура', topn=5) 

[('деятель', 0.845112144947052),
 ('народный', 0.8272836208343506),
 ('наука', 0.8048849701881409),
 ('наследие', 0.7851907014846802),
 ('культурный', 0.7830075621604919)]

In [100]:
w2v.wv.most_similar('наука', topn=5) 

[('зеф', 0.8651098608970642),
 ('деятель', 0.8504012823104858),
 ('культурный', 0.8341472148895264),
 ('спрайч', 0.8288112282752991),
 ('каракалпакстан', 0.8273425102233887)]

In [101]:
w2v.wv.most_similar('экономика', topn=5) 

[('инфляция', 0.8582209944725037),
 ('кризис', 0.8557974100112915),
 ('рецессия', 0.8554232716560364),
 ('замедление', 0.8542715311050415),
 ('рост', 0.8527257442474365)]

In [102]:
w2v.wv.most_similar('бизнес', topn=5) 

[('инвестиционный', 0.8342058658599854),
 ('отрасль', 0.8306347727775574),
 ('корпоративный', 0.8264024257659912),
 ('сфера', 0.8177992105484009),
 ('инвестор', 0.7992810010910034)]

In [103]:
w2v.wv.most_similar('жизнь', topn=5) 

[('случаться', 0.7522169351577759),
 ('болезнь', 0.7465179562568665),
 ('существование', 0.734734058380127),
 ('родитель', 0.7345521450042725),
 ('жить', 0.7316579818725586)]

In [104]:
w2v.wv.most_similar('путешествие', topn=5) 

[('путешествовать', 0.8229612708091736),
 ('замёрзлый', 0.8202993869781494),
 ('мэрфь', 0.8197442889213562),
 ('сеанс', 0.8173333406448364),
 ('долгий', 0.804823100566864)]

Реализовать алгоритм классификации документа по категориям, посчитать точноть на тестовых данных, подобрать гиперпараметры. Метод векторизации выбрать произвольно - можно использовать  𝑡𝑓−𝑖𝑑𝑓  с понижением размерности (см. scikit-learn), можно использовать обученные на предыдущем шаге векторные представления, можно использовать предобученные модели. Имейте ввиду, что простое "усреднение" токенов в тексте скорее всего не даст положительных результатов. Нужно реализовать два алгоритмов из трех:
1. SVM
2. наивный байесовский классификатор
3. логистическая регрессия

In [130]:
from sklearn.svm import LinearSVC
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import GridSearchCV
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy.stats import uniform

In [106]:
vectorizer = TfidfVectorizer(min_df = 10, max_df = 1., ngram_range = (1,2), max_features = 1000)

In [140]:
train_tfidf = vectorizer.fit_transform(train)
test_tfidf = vectorizer.transform(test)

In [141]:
train_labels = np.array(train_data['target'])
test_labels = np.array(test_data['target'])

In [142]:
distributions = dict(alpha=uniform(loc=0, scale=1.5))
naive_bayes = RandomizedSearchCV(MultinomialNB(), distributions)
search = naive_bayes.fit(train_tfidf, train_labels)

In [143]:
pred = search.predict(test_tfidf)
print('NB accuracy: ', accuracy_score(pred, test_labels))

NB accuracy:  0.7303333333333333


In [145]:
distributions = {'C': [1, 10, 100], 'gamma': [0.01, 0.1, 1],'kernel': ['linear', 'sigmoid']}
svm =  GridSearchCV(SVC(), distributions, cv=2, verbose=3)
search = svm.fit(train_tfidf, train_labels)
 

Fitting 2 folds for each of 18 candidates, totalling 36 fits
[CV 1/2] END ....C=1, gamma=0.01, kernel=linear;, score=0.840 total time=  45.3s
[CV 2/2] END ....C=1, gamma=0.01, kernel=linear;, score=0.834 total time=  43.2s
[CV 1/2] END ...C=1, gamma=0.01, kernel=sigmoid;, score=0.509 total time= 1.8min
[CV 2/2] END ...C=1, gamma=0.01, kernel=sigmoid;, score=0.534 total time= 1.8min
[CV 1/2] END .....C=1, gamma=0.1, kernel=linear;, score=0.840 total time=  43.1s
[CV 2/2] END .....C=1, gamma=0.1, kernel=linear;, score=0.834 total time=  43.3s
[CV 1/2] END ....C=1, gamma=0.1, kernel=sigmoid;, score=0.805 total time= 1.1min
[CV 2/2] END ....C=1, gamma=0.1, kernel=sigmoid;, score=0.802 total time= 1.1min
[CV 1/2] END .......C=1, gamma=1, kernel=linear;, score=0.840 total time=  42.7s
[CV 2/2] END .......C=1, gamma=1, kernel=linear;, score=0.834 total time=  43.4s
[CV 1/2] END ......C=1, gamma=1, kernel=sigmoid;, score=0.836 total time=  41.0s
[CV 2/2] END ......C=1, gamma=1, kernel=sigmoid;

In [146]:
pred = search.predict(test_tfidf)
print('SVM: ', accuracy_score(pred, test_labels))

SVM:  0.641
